# How to validate a model on chronologically ordered data which also contains groups?

Since it takes quite some time to get a utility (leaderboard) score back for our model, it would be nice to be able to 'locally' calculate an indication of a model's performance; independent of the (time expensive and limited) submission API. This would allow for much better tuning of hyper-parameters or other aspects of the model's training process.

In this notebook I want to lay out a couple of techniques that can be used to do this. For every step we will see that there is a problem with using it for this particular competition. Fortunately the last chapter provides a solution! If you are not interested in an introduction in test and validation techniques, then skip to the bottom. First up: train and test subsets.

## Train and Test Subsets

The first obvious step is to set apart some data which the model never gets to see. After the model has been trained, we use that unseen data to verify our model's predicitions. Scikit-learn's [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) makes the process of splitting datasets easy for us. Let's load our training data and set aside a test set:

In [1]:
import numpy as np
import pandas as pd

try:
    dtrain = pd.read_parquet('../input/found-the-holy-grail-grouptimeseriessplit/dtrain.parquet')
except:
    dtrain = pd.read_csv('../input/jane-street-market-prediction/train.csv', index_col='ts_id')
    dtrain = dtrain.astype({c: np.float32 for c, t in dtrain.dtypes.items() if t == np.float64})
    #dtrain.to_parquet('dtrain.parquet')

In [2]:
dlabels = dtrain.filter(regex='resp')
dtrain = dtrain.drop(dlabels.columns, axis=1)

print(dtrain.columns)
print(dlabels.columns)

Index(['date', 'weight', 'feature_0', 'feature_1', 'feature_2', 'feature_3',
       'feature_4', 'feature_5', 'feature_6', 'feature_7',
       ...
       'feature_120', 'feature_121', 'feature_122', 'feature_123',
       'feature_124', 'feature_125', 'feature_126', 'feature_127',
       'feature_128', 'feature_129'],
      dtype='object', length=132)
Index(['resp_1', 'resp_2', 'resp_3', 'resp_4', 'resp'], dtype='object')


In [3]:
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(
    dtrain,
    dlabels,
    test_size=.25,
    random_state=1,
    shuffle=False
)

print(x_train.shape, x_test.shape)
print(x_train.index)

(1792868, 132) (597623, 132)
Int64Index([      0,       1,       2,       3,       4,       5,       6,
                  7,       8,       9,
            ...
            1792858, 1792859, 1792860, 1792861, 1792862, 1792863, 1792864,
            1792865, 1792866, 1792867],
           dtype='int64', name='ts_id', length=1792868)


Use of `shuffle=False` is key here; since otherwise we would lose all chronological order.

However, this approach is problematic because by constantly verifying on the same data, we also slowly start to overfit on the test set ("leakage"). Splitting the test set again into a validation set could solve this: the model's hyper-parameters are tuned and verified on the validation set and once that is completely finished we test it (only once!) on the test set. The problem now becomes that either the test and validation sets become too small to be useful or that so much data is used to validate and test that nog enough data remains to train on.

## Cross-validation

In cross-validation (CV), multiple validation sets are derived from the training set. Every *fold* a new part of the training set is used as the vaildation set, and the data previously used for validation now becomes part of the training set again:

![img](https://scikit-learn.org/stable/_images/grid_search_cross_validation.png)

(Source: [scikit-learn's User Guide, Ch. 3.1](https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-evaluating-estimator-performance).)

In [4]:
from sklearn.model_selection import KFold

for train_idx, test_idx in KFold().split(x_train):
    #print(train, test)
    print(x_train.loc[train_idx, 'date'].index)
    print(x_train.loc[test_idx, 'date'].index)
    break

Int64Index([ 358574,  358575,  358576,  358577,  358578,  358579,  358580,
             358581,  358582,  358583,
            ...
            1792858, 1792859, 1792860, 1792861, 1792862, 1792863, 1792864,
            1792865, 1792866, 1792867],
           dtype='int64', name='ts_id', length=1434294)
Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,
            ...
            358564, 358565, 358566, 358567, 358568, 358569, 358570, 358571,
            358572, 358573],
           dtype='int64', name='ts_id', length=358574)


### TimeSeriesSplit

Note however, that in the first split shown above we are validating our *chronological* data on the past. We are training on trades starting from `358574` but testing on trades starting from `0`. In other words, our model has been trained using information which wasn't yet available at the time of the validation set. This is clear leakage; we are predicting the past with knowledge from the future. But our aim is to predict data in the future! This problem has already been addressed by scikit-learn in the form of [TimeSeriesSplit](https://scikit-learn.org/stable/modules/cross_validation.html#time-series-split):
![](https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_0101.png)
But what is the problem this time? `TimeSeriesSplit` does not respect the groups available in the data. Although not clearly visible in this plot, we can imagine that a group can partially fall in the training set and partially in the test set.

In [5]:
from sklearn.model_selection import TimeSeriesSplit

for train_idx, test_idx in TimeSeriesSplit(n_splits=5).split(x_train):
    print(x_train.loc[train_idx, 'date'].unique())
    print(x_train.loc[test_idx, 'date'].unique())

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44]
[ 44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61
  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79
  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97
  98  99 100 101 102 103 104]
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104]
[104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121
 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139
 140 141 142 143 144 145 146 147 148 149

Already in the first split we can see data from day `44` present in both the training and test set. That would mean that we are training on half of the trades of a certain day, just to validate their performance on the other half of the trades of that day. What we of course want is to train on all trades of a particular day, and to validate them on the day that follows! Otherwise again leaking will occur.

### GroupKFold

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_0051.png)

In [6]:
from sklearn.model_selection import GroupKFold

for train_idx, test_idx in GroupKFold().split(x_train, groups=x_train['date']):
    print(x_train.loc[train_idx, 'date'].unique())
    print(x_train.loc[test_idx, 'date'].unique())
    break

[  0   1   2   3   4   9  10  11  12  13  14  15  16  17  18  19  20  21
  22  23  24  25  26  27  29  30  32  33  34  35  36  37  38  39  40  41
  42  43  45  47  48  49  50  52  53  54  55  56  57  58  59  60  61  62
  63  64  65  66  67  68  69  71  72  73  75  76  77  78  79  82  83  84
  85  86  87  88  89  90  91  92  93  95  96  97  98  99 100 101 102 103
 104 105 106 107 108 109 110 111 112 113 116 117 121 122 123 124 125 126
 127 129 130 131 132 134 135 136 137 138 139 140 141 142 143 144 147 148
 149 150 151 152 153 154 155 156 158 159 160 161 162 163 164 165 166 168
 169 171 173 174 175 176 178 179 180 181 182 183 184 185 188 190 191 192
 193 194 195 196 199 200 202 203 206 208 209 210 211 212 213 214 215 216
 217 219 220 221 222 223 226 227 230 231 232 233 235 236 237 238 239 240
 241 244 248 249 250 251 252 253 254 255 256 258 259 260 261 263 265 266
 268 269 270 271 272 273 274 275 276 278 279 280 282 283 284 286 287 288
 289 290 291 292 297 298 299 300 301 302 303 304 30

The [GroupKFold](https://scikit-learn.org/stable/modules/cross_validation.html#group-k-fold) iterator does respect groupings: no group will ever be part of two folds. Unfortunately, it is also clear that it mixes up the order completely and thus loses the temporal dimension again. What we need is a a crossover between `GroupKFold` and `TimeSeriesSplit`: `GroupTimesSeriesSplit`.

### GroupTimesSeriesSplit

OK, so this iterator does not exist yet in scikit-learn. However, a request for it has been documented on GitHub over a year ago ([Feature request: Group aware Time-based cross validation #14257](https://github.com/scikit-learn/scikit-learn/issues/14257)) and is almost ready for release. Thanks to open source we can take a sneak peek already! 

**Do note that this is not fully reviewed yet!!!** This might be the final code that it will make it into `sklearn`'s version `0.24` as a major feature, but there's also a chance of bugs still being present.

I did not write *any* of this but it did take me a good day of research and trying to write it myself. All credits go to [@getgaurav2](https://github.com/getgaurav2/).

Here are some more attempts at grouped cross-validation I encountered in my research:
- https://stackoverflow.com/questions/51963713/cross-validation-for-grouped-time-series-panel-data
- https://datascience.stackexchange.com/questions/77684/time-series-grouped-cross-validation
- https://nander.cc/writing-custom-cross-validation-methods-grid-search

In [7]:
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class GroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_size : int, default=None
        Maximum size for a single training set.
    Examples
    --------
    >>> import numpy as np
    >>> from sklearn.model_selection import GroupTimeSeriesSplit
    >>> groups = np.array(['a', 'a', 'a', 'a', 'a', 'a',\
                           'b', 'b', 'b', 'b', 'b',\
                           'c', 'c', 'c', 'c',\
                           'd', 'd', 'd'])
    >>> gtss = GroupTimeSeriesSplit(n_splits=3)
    >>> for train_idx, test_idx in gtss.split(groups, groups=groups):
    ...     print("TRAIN:", train_idx, "TEST:", test_idx)
    ...     print("TRAIN GROUP:", groups[train_idx],\
                  "TEST GROUP:", groups[test_idx])
    TRAIN: [0, 1, 2, 3, 4, 5] TEST: [6, 7, 8, 9, 10]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a']\
    TEST GROUP: ['b' 'b' 'b' 'b' 'b']
    TRAIN: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] TEST: [11, 12, 13, 14]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a' 'b' 'b' 'b' 'b' 'b']\
    TEST GROUP: ['c' 'c' 'c' 'c']
    TRAIN: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]\
    TEST: [15, 16, 17]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a' 'b' 'b' 'b' 'b' 'b' 'c' 'c' 'c' 'c']\
    TEST GROUP: ['d' 'd' 'd']
    """
    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_size=None
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_size = max_train_size

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))
        group_test_size = n_groups // n_folds
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []
            for train_group_idx in unique_groups[:group_test_start]:
                train_array_tmp = group_dict[train_group_idx]
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)
            train_end = train_array.size
            if self.max_train_size and self.max_train_size < train_end:
                train_array = train_array[train_end -
                                          self.max_train_size:train_end]
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)
            yield [int(i) for i in train_array], [int(i) for i in test_array]

In [8]:
for train_idx, test_idx in GroupTimeSeriesSplit().split(x_train, groups=x_train['date']):
    print(x_train.loc[train_idx, 'date'].unique())
    print(x_train.loc[test_idx, 'date'].unique())

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67]
[ 68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85
  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101 102 103
 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121
 122 123 124 125 126 127 128 129 130]
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130]
[1

Chronological order is maintained and group separation is respected!

In [9]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# modified code for group gaps; source
# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size
 
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]
            
            
            if self.verbose > 0:
                    pass
                    
            yield [int(i) for i in train_array], [int(i) for i in test_array]

In [10]:
for train_idx, test_idx in PurgedGroupTimeSeriesSplit(n_splits = 5, group_gap = 31).split(x_train, groups=x_train['date']):
    print(x_train.loc[train_idx, 'date'].unique())
    print(x_train.loc[test_idx, 'date'].unique())

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36]
[ 68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85
  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101 102 103
 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121
 122 123 124 125 126 127 128 129 130]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]
[131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148
 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166
 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184
 185 186 187 188 189 190 191 192 193]
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  